In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 7.9 MB/s eta 0:00:00:00:01


In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("omdabral/indian-penal-code-complete-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/indian-penal-code-complete-dataset


In [2]:
import pandas as pd
df = pd.read_csv("/kaggle/input/indian-penal-code-complete-dataset/FIR_DATASET.csv")

In [4]:
df.head()

,URL,Description,Offense,Punishment,Cognizable,Bailable,Court
0,https://lawrato.com/indian-kanoon/ipc/section-140,Description of IPC Section 140\nAccording to s...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,Cognizable,Bailable,Any Magistrate
1,https://lawrato.com/indian-kanoon/ipc/section-127,Description of IPC Section 127\nAccording to s...,Receiving property taken by war or depredation...,7 Years + Fine + forfeiture of property,Cognizable,Non-Bailable,Court of Session
2,https://lawrato.com/indian-kanoon/ipc/section-128,Description of IPC Section 128\nAccording to s...,Public servant voluntarily allowing prisoner o...,Imprisonment for Life or 10 Years + Fine,Cognizable,Non-Bailable,Court of Session
3,https://lawrato.com/indian-kanoon/ipc/section-129,Description of IPC Section 129\nAccording to s...,Public servant negligently suffering prisoner ...,Simple Imprisonment 3 Years + Fine,Cognizable,Bailable,Magistrate First Class
4,https://lawrato.com/indian-kanoon/ipc/section-130,Description of IPC Section 130\nAccording to s...,"Aiding escape of, rescuing or harbouring, such...",Imprisonment for Life or 10 Years + Fine,Cognizable,Non-Bailable,Court of Session


In [5]:
df.drop("URL", axis=1, inplace=True) 

In [6]:
df.head()

,Description,Offense,Punishment,Cognizable,Bailable,Court
0,Description of IPC Section 140\nAccording to s...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,Cognizable,Bailable,Any Magistrate
1,Description of IPC Section 127\nAccording to s...,Receiving property taken by war or depredation...,7 Years + Fine + forfeiture of property,Cognizable,Non-Bailable,Court of Session
2,Description of IPC Section 128\nAccording to s...,Public servant voluntarily allowing prisoner o...,Imprisonment for Life or 10 Years + Fine,Cognizable,Non-Bailable,Court of Session
3,Description of IPC Section 129\nAccording to s...,Public servant negligently suffering prisoner ...,Simple Imprisonment 3 Years + Fine,Cognizable,Bailable,Magistrate First Class
4,Description of IPC Section 130\nAccording to s...,"Aiding escape of, rescuing or harbouring, such...",Imprisonment for Life or 10 Years + Fine,Cognizable,Non-Bailable,Court of Session


In [8]:
import pandas as pd
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nlpaueb/legal-bert-base-uncased")  
flattened_chunks = df.apply(lambda row: " ".join(row[["Description", "Offense", "Punishment"]].astype(str)), axis=1).tolist()

embeddings = model.encode(flattened_chunks, convert_to_numpy=True)
vectors = [
    {
        "id": f"vec{i}",
        "values": emb.tolist(),  # Convert NumPy array to list
        "metadata": {
            "Description": row["Description"],
            "Offense": row["Offense"],
            "Punishment": row["Punishment"],
            "Cognizable": row["Cognizable"],
            "Bailable": row["Bailable"],
            "Court": row["Court"]
        }
    }
    for i, (row, emb) in enumerate(zip(df.to_dict(orient="records"), embeddings))
]

print(vectors[0]) 


Batches:   0%|          | 0/14 [00:00<?, ?it/s]

{'id': 'vec0', 'values': [-0.06380651146173477, 0.40495991706848145, 0.13302575051784515, -0.16680790483951569, 0.4149867296218872, -0.22514694929122925, 0.243968665599823, 0.5072693228721619, -0.3008967638015747, -0.13719627261161804, 0.19097745418548584, -0.019738079980015755, 0.058631379157304764, 0.08990478515625, -0.30962082743644714, 0.34368523955345154, 0.20893451571464539, -0.015438822098076344, 0.12709477543830872, 0.005971909966319799, -0.09175646305084229, 0.0608176626265049, -0.05013103783130646, 0.22756966948509216, 0.3854771852493286, 0.24563242495059967, 0.020763106644153595, -0.1829715073108673, 0.01919352449476719, -0.26662763953208923, 0.0218043252825737, 0.02722487971186638, -0.1041727364063263, -0.25206494331359863, 0.1528717577457428, 0.08105891197919846, -0.03399679809808731, -0.0885772556066513, 0.16134744882583618, 0.19568362832069397, -0.18487243354320526, -0.3406292796134949, 0.06238598749041557, 9.508378570899367e-05, -0.2866552472114563, -0.15170885622501373

In [11]:
from pinecone import Pinecone
from tqdm import tqdm

pc = Pinecone("pcsk_3eb3fw_R8LSeRPSZyQpcLpMsMZ6iG3LFN1DUAbbUs7WBend67X4ocTzdCbe8ovBHALB4gb")
index = pc.Index("ipc")

batch_size = 5  

for i in tqdm(range(0, len(vectors), batch_size), desc="Uploading batches"):
    batch = vectors[i:i+batch_size]
    index.upsert(vectors=batch)


Uploading batches:   7%|▋         | 6/89 [00:01<00:23,  3.58it/s]


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 13 Feb 2025 17:14:25 GMT', 'Content-Type': 'text/plain', 'Content-Length': '80', 'Connection': 'keep-alive', 'server': 'envoy'})
HTTP response body: Unexpected token.
ffray.", "Offense": NaN, "Punishment": N
                    ^


In [12]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from pinecone import Pinecone

model = SentenceTransformer("nlpaueb/legal-bert-base-uncased")  # Change model if needed

# Flatten only 'Description', 'Offense', and 'Punishment' columns into a single string
flattened_chunks = df.apply(lambda row: " ".join(row[["Description", "Offense", "Punishment"]].fillna("").astype(str)), axis=1).tolist()

# Get embeddings
embeddings = model.encode(flattened_chunks, convert_to_numpy=True)

# Create vector format with proper metadata handling
vectors = [
    {
        "id": f"vec{i}",
        "values": emb.tolist(),  # Convert NumPy array to list
        "metadata": {
            "Description": str(row["Description"]),
            "Offense": str(row["Offense"]),
            "Punishment": str(row["Punishment"]),
            "Cognizable": str(row["Cognizable"]),
            "Bailable": str(row["Bailable"]),
            "Court": str(row["Court"])
        }
    }
    for i, (row, emb) in enumerate(zip(df.fillna("").to_dict(orient="records"), embeddings))
]

# Example Output
print(vectors[0])  # Print first vector

# Initialize Pinecone (Use environment variables or config file for security)
pc = Pinecone("pcsk_3eb3fw_R8LSeRPSZyQpcLpMsMZ6iG3LFN1DUAbbUs7WBend67X4ocTzdCbe8ovBHALB4gb")  # Replace with your API key securely
index = pc.Index("ipc")

# Define batch size
batch_size = 5  # Adjust as needed

# Upsert in batches with tqdm for progress tracking
for i in tqdm(range(0, len(vectors), batch_size), desc="Uploading batches"):
    batch = vectors[i:i+batch_size]
    try:
        index.upsert(vectors=batch)
    except Exception as e:
        print(f"Error in batch {i}: {e}")  # Catch and display errors for debugging


Batches:   0%|          | 0/14 [00:00<?, ?it/s]

{'id': 'vec0', 'values': [-0.06380651146173477, 0.40495991706848145, 0.13302575051784515, -0.16680790483951569, 0.4149867296218872, -0.22514694929122925, 0.243968665599823, 0.5072693228721619, -0.3008967638015747, -0.13719627261161804, 0.19097745418548584, -0.019738079980015755, 0.058631379157304764, 0.08990478515625, -0.30962082743644714, 0.34368523955345154, 0.20893451571464539, -0.015438822098076344, 0.12709477543830872, 0.005971909966319799, -0.09175646305084229, 0.0608176626265049, -0.05013103783130646, 0.22756966948509216, 0.3854771852493286, 0.24563242495059967, 0.020763106644153595, -0.1829715073108673, 0.01919352449476719, -0.26662763953208923, 0.0218043252825737, 0.02722487971186638, -0.1041727364063263, -0.25206494331359863, 0.1528717577457428, 0.08105891197919846, -0.03399679809808731, -0.0885772556066513, 0.16134744882583618, 0.19568362832069397, -0.18487243354320526, -0.3406292796134949, 0.06238598749041557, 9.508378570899367e-05, -0.2866552472114563, -0.15170885622501373

Uploading batches: 100%|██████████| 89/89 [00:13<00:00,  6.46it/s]


In [15]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone

model = SentenceTransformer("nlpaueb/legal-bert-base-uncased")

user_query = "Punishment for stealing, taking someone's property without permission, unlawfully taking possession, criminal offense of theft, legal consequences of stealing"

query_embedding = model.encode(user_query, convert_to_numpy=True).tolist()  # Convert to list for Pinecone

pc = Pinecone("pcsk_3eb3fw_R8LSeRPSZyQpcLpMsMZ6iG3LFN1DUAbbUs7WBend67X4ocTzdCbe8ovBHALB4gb")  # Replace securely
index = pc.Index("ipc")

search_results = index.query(vector=query_embedding, top_k=5, include_metadata=True)  # Retrieve top 5 results
for match in search_results["matches"]:
    print(f"Score: {match['score']}")
    print(f"Offense: {match['metadata']['Offense']}")
    print(f"Description: {match['metadata']['Description']}")
    print(f"Punishment: {match['metadata']['Punishment']}")
    print("-" * 50)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Score: 0.905988514
Offense: 
Description: Description of IPC Section 378
According to section 378 of Indian penal code, Whoever, intending to take dishonestly any movable property out of the possession of any person without that personâ€™s consent, moves that property in order to such taking, is said to commit theft.


IPC 378 in Simple Words
In the Indian Penal Code, moving someone's movable property with the intention of dishonestly taking it without their consent constitutes theft.
Punishment: 
--------------------------------------------------
Score: 0.896423042
Offense: Assault or criminal force in attempt to commit theft of property worn or carried by a person
Description: Description of IPC Section 356
According to section 356 of Indian penal code, Whoever assaults or uses criminal force to any person, in attempting to commit theft on any property which that person is then wearing or carrying shall be punished with imprisonment of either description for a term which may extend t